In [28]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy
import os
import glob

In [9]:
import pandas as pd

In [10]:
ls

fastai@                                             test.tsv
movie-review-sentiment-analysis-kernels-only.ipynb  train.tsv
sampleSubmission.csv


In [11]:
df = pd.read_csv("train.tsv",sep="\t")

In [12]:
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [13]:
df_sen = pd.DataFrame()
df_sen["sentence"] = 0
sentences = []
for i in df["SentenceId"].unique():  
    sentences.append([df[df["SentenceId"] == i].reset_index().loc[0]["Phrase"],df[df["SentenceId"] == i].reset_index().loc[0]["Sentiment"]])
#     print(df[i].reset_index().loc[0]["Phrase"])
df_sen = pd.DataFrame(sentences)

In [14]:
df_sen[:10]

,0,1
0,A series of escapades demonstrating the adage ...,1
1,"This quiet , introspective and entertaining in...",4
2,"Even fans of Ismail Merchant 's work , I suspe...",1
3,A positively thrilling combination of ethnogra...,3
4,Aggressive self-glorification and a manipulati...,1
5,A comedy-drama of nearly epic proportions root...,4
6,"Narratively , Trouble Every Day is a plodding ...",1
7,"The Importance of Being Earnest , so thick wit...",3
8,But it does n't leave you with much .,1
9,You could hate it for the same reason .,1


In [12]:
!mkdir input_txt_folder

In [7]:
fname = os.path.join("test","file")+".txt"
fname

'test/file.txt'

In [17]:
count = 1
for index, row in df_sen.iterrows():
    phrase=row[0]
    label = row[1]
    filename = os.path.join("labeled_text", str(label), str(count))+".txt"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename,"w") as f:
        f.write(phrase)
    count = count + 1

In [18]:
#total number of sentences
count

8530

In [32]:
#Show labels
labelList = !ls labeled_text
labelList

['0', '1', '2', '3', '4']

In [38]:
#Count number of samples under each label
labelCount = []
for i in labelList:
    trn_files = !ls labeled_text/{i}
    labelCount.append(len(trn_files))
labelCount

[1072, 2200, 1655, 2321, 1281]

In [31]:
sample = !cat input_txt_folder/2.txt
sample

['This quiet , introspective and entertaining independent is worth seeking .']

In [32]:
!find input_txt_folder -name '*.txt' | xargs cat | wc -w

153518


In [50]:
class MovieReview(torchtext.data.Dataset):
    """Create a MovieReview dataset instance given a path and fields.
    Arguments:
        path: Path to the dataset's highest level directory
        text_field: The field that will be used for text data.
        label_field: The field that will be used for label data.
        Remaining keyword arguments: Passed to the constructor of
            data.Dataset.
    """
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['0','1','2','3','4']:
            fnames = glob.glob(os.path.join(path, label, "*.txt"))
            assert fnames, f"can't find file under {path}/{label}"
            for fname in fnames:
                with open(fname, "r") as f: text = f.readline()
                examples.append(data.Example.fromlist([text,label], fields))
        super().__init__(examples, fields, **kwargs)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data', 
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)
    
        

In [45]:
MovieReviewLabel = data.Field(sequential=False)

In [47]:
TEXT = pickle.load(open("TEXT.pkl","rb")) #open in read and bineary mode

In [48]:
PATH = "labeled_text"

In [52]:
splits = MovieReview.splits(TEXT, MovieReviewLabel, PATH,train="")

AssertionError: can't find file under labeled_text/test/0